In [1]:
import pandas as pd 
df = pd.read_csv('Data_drop_year.csv')

In [2]:
train = df['Comment']
# drop the six fist columns
test = df.drop(df.columns[0:6], axis=1)

In [3]:
# split the data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, test, test_size=0.2, random_state=42)

# tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,2),max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [4]:
y_test

,'data integration','marketing and communication','technology','payment and checkout','shopping experience','unemployment','product available and store design','price and value','general','privacy and security'
5764,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
62280,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
37940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
71238,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
20330,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
84292,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
26956,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
58966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
41187,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# create a model for multi-label classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
parameters = {'estimator__max_depth': [2,3,4, 5,6, 7,8,9, 10, 20, 30, 40, 50, 60, None]
                ,'estimator__criterion': ['entropy']
                ,'estimator__max_features': ['sqrt', 'auto']
                ,'estimator__min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]
                ,'estimator__min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
                ,'estimator__min_weight_fraction_leaf': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
clf_gv = GridSearchCV(OneVsRestClassifier(DecisionTreeClassifier()), parameters, cv=3,scoring='accuracy',n_jobs=-1)

# fit the model
clf_gv.fit(X_train_tfidf, y_train)

# print the best parameters
print(clf_gv.best_params_)
print(clf_gv.best_score_)
print(clf_gv.best_estimator_)

c:\Python310\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
32400 fits failed out of a total of 81000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8100 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python310\lib\site-packages\sklearn\multiclass.py", line 330, in fit
    sel

{'estimator__criterion': 'entropy', 'estimator__max_depth': None, 'estimator__max_features': 'auto', 'estimator__min_samples_leaf': 4, 'estimator__min_samples_split': 10, 'estimator__min_weight_fraction_leaf': 0.0}
0.587925723087455
OneVsRestClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                     max_features='auto',
                                                     min_samples_leaf=4,
                                                     min_samples_split=10))


In [9]:
from sklearn.model_selection import cross_validate
#import decision tree classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
# onevsrestclassifier with decision tree classifier with best parameters
clf_tree = OneVsRestClassifier(DecisionTreeClassifier(criterion='entropy',  max_features= 'auto', min_samples_leaf= 4, min_samples_split= 10, min_weight_fraction_leaf= 0.0), n_jobs=-1)


In [10]:
# print accuracy
from sklearn.metrics import accuracy_score
clf_tree.fit(X_train_tfidf, y_train)
y_pred = clf_tree.predict(X_test_tfidf)
print(accuracy_score(y_test, y_pred))
    

0.5949229891614375
